## Initialize your OpenSearchDocumentStore

You can use `opensearch_indexing_pipeline.py` or the `notebooks/opensearch_indexing_pipeline.ipynb` notebook for a step by step guide to index the example files to your own `OpenSearchDocumentStore`. You may do this locally, or deploy it on AWS. Depending on your setup, once you have a running DocumentStore, connect to it in the cell below by providing the right credentials to `host`, `port`, `username` and `password`.

In [ ]:
from haystack.document_stores import OpenSearchDocumentStore

doc_store = OpenSearchDocumentStore(host='your_opensearch_host', port=443, username= "admin", password="admin", embedding_dim=384)

## Initialize a PromptNode with your SageMaker Endpoint Credentials

Once you've deployed your model on SageMaker provide your own credentials in `model_name_or_path`, `profile_name` and `region_name`

In [6]:
from haystack.nodes import AnswerParser, EmbeddingRetriever, PromptNode, PromptTemplate

question_answering = PromptTemplate(prompt="Given the context please answer the question. If the answer is not contained within the context below, say 'I don't know'.\n" 
                                            "Context: {join(documents)};\n Question: {query};\n Answer: ", output_parser=AnswerParser(reference_pattern=r"Document\[(\d+)\]"))

gen_qa_with_references = PromptNode(default_prompt_template=question_answering,  model_name_or_path="jumpstart-dft-falcon-40b-instruct", model_kwargs={"aws_profile_name": "default", "aws_region_name": "us-east-1"})


In [ ]:
retriever = EmbeddingRetriever(document_store=doc_store, embedding_model="sentence-transformers/all-MiniLM-L12-v2", devices=["mps"], top_k=5)

## Create a retrieval-augmented QA pipeline

In [8]:
from haystack import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=['Query'])
pipe.add_node(component=gen_qa_with_references, name="GenQAWithRefPromptNode", inputs=["Retriever"])

In [ ]:
from haystack.utils import print_answers

result = pipe.run("How do I install the opensearch cli?", params={"Retriever":{"top_k": 3}})

In [ ]:
print_answers(results=result, details="minimum")